# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df=pd.read_csv('citydata.csv')
city_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed
0,cn,yuancheng,36.6520,107.7896,36.30,36.30,63,0,17.85
1,ph,sapa,5.1544,120.3241,83.57,83.57,75,75,5.75
2,sh,jamestown,42.0970,-79.2353,34.93,36.00,80,90,5.75
3,ca,pangnirtung,66.1451,-65.7125,-5.21,-5.21,76,100,0.63
4,ki,butaritari,3.0707,172.7902,81.41,81.41,80,82,18.68
...,...,...,...,...,...,...,...,...,...
692,ru,sladkovo,55.5284,70.3385,-11.04,-11.04,87,0,4.94
693,cl,tocopilla,-22.0920,-70.1979,65.91,65.91,68,72,5.37
694,es,nijar,36.9665,-2.2060,42.80,42.80,75,0,3.44
695,pw,kloulklubed,7.0419,134.2556,82.40,82.40,83,90,6.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lnt"]].astype(float)
humidity=city_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#& city_df.Wind_Speed.isin([0, 9])

new_df=city_df.loc[city_df['Max_Temp']<80]
new_df=city_df.loc[city_df['Max_Temp']>80]
new_df=city_df.loc[city_df['Wind_Speed']<10]
new_df=city_df.loc[city_df['Cloudiness']==0]
new_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed
0,cn,yuancheng,36.6520,107.7896,36.30,36.30,63,0,17.85
11,gh,kintampo,8.0563,-1.7306,76.86,76.86,66,0,3.56
30,ru,tavda,58.0425,65.2726,-8.50,-8.50,87,0,5.86
33,gl,qaanaaq,77.4840,-69.3632,-1.86,-1.86,75,0,4.38
52,no,eidsvag,62.7767,8.0668,18.50,21.20,79,0,5.75
...,...,...,...,...,...,...,...,...,...
673,uy,rocha,-34.4833,-54.3333,54.30,54.30,81,0,3.09
682,tv,asau,46.4333,26.4000,27.25,30.00,93,0,8.05
690,et,mega,4.0500,38.3000,74.16,74.16,26,0,18.75
692,ru,sladkovo,55.5284,70.3385,-11.04,-11.04,87,0,4.94


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# find the closest restaurant of each type to coordinates
radius=5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "lodging","key": g_key,"radius":radius}
#new_df.loc[:,"Hotel_Name"]=""
#new_df.loc[:,"Hotel_Address"]=""
#new_df.loc[:,"Hotel_Lat"]=""
#new_df.loc[:,"Hotel_Lnt"]=""
hotel_df=new_df
# use iterrows to iterate through pandas dataframe

for index, row in new_df.iterrows():

    # get restaurant type from df
    location_lat = str(round(row['Lat'],6))
    location_lnt = str(round(row['Lnt'],6))
    params['location']=location_lat+","+location_lnt
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest Hotel is {results[0]['name']}.")        
        hotel_df.loc[index,'Hotel name']=results[0]['name']
        hotel_df.loc[index,'Hotel_Address'] = results[0]['vicinity']
        hotel_df.loc[index,'Hotel_Lat']=results[0]['geometry']['location']['lat']
        hotel_df.loc[index,'Hotel_Lnt']=results[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results for Index 0.
Missing field/result... skipping.
Retrieving Results for Index 11.
Closest Hotel is Adomaa Memorial Hotel.
Retrieving Results for Index 30.


C:\Users\Oscar\.conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Oscar\.conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest Hotel is The hotel is number 1.
Retrieving Results for Index 33.
Closest Hotel is Qaanaaq Hotel.
Retrieving Results for Index 52.
Closest Hotel is Eidsvåg Fjordhotell.
Retrieving Results for Index 62.
Closest Hotel is Friendship Hotel.
Retrieving Results for Index 66.
Missing field/result... skipping.
Retrieving Results for Index 71.
Closest Hotel is Kingsgate Hotel Te Anau.
Retrieving Results for Index 73.
Closest Hotel is Erzscheidergården.
Retrieving Results for Index 74.
Closest Hotel is Chambres d'hôte "La Trinité".
Retrieving Results for Index 88.
Closest Hotel is Hotel y restaurante "El Diamante".
Retrieving Results for Index 98.
Closest Hotel is The Yellow House.
Retrieving Results for Index 103.
Missing field/result... skipping.
Retrieving Results for Index 105.
Closest Hotel is Bjarnabúð.
Retrieving Results for Index 106.
Closest Hotel is Royal Exchange Hotel.
Retrieving Results for Index 113.
Closest Hotel is Lic premium point.
Retrieving Results for Index 126.
Close

Closest Hotel is Hua Ling Grand Hotel Urumqi.
Retrieving Results for Index 645.
Closest Hotel is Auto Hotel El Soñador.
Retrieving Results for Index 648.
Missing field/result... skipping.
Retrieving Results for Index 654.
Closest Hotel is Wangjiang Building.
Retrieving Results for Index 667.
Closest Hotel is محل الشريف محمد العايب.
Retrieving Results for Index 670.
Closest Hotel is Turkwel Lodge.
Retrieving Results for Index 673.
Closest Hotel is Beleza Pura B&B.
Retrieving Results for Index 682.
Closest Hotel is Casa Mika Ciobanus.
Retrieving Results for Index 690.
Closest Hotel is Amanuel Guest House.
Retrieving Results for Index 692.
Closest Hotel is Uyutnaya.
Retrieving Results for Index 694.
Closest Hotel is La Casa del Cine.


In [7]:
hotel_df

,Country,City,Lat,Lnt,Temperature,Max_Temp,Humidity,Cloudiness,Wind_Speed,Hotel name,Hotel_Address,Hotel_Lat,Hotel_Lnt
0,cn,yuancheng,36.6520,107.7896,36.30,36.30,63,0,17.85,NaN,NaN,NaN,NaN
11,gh,kintampo,8.0563,-1.7306,76.86,76.86,66,0,3.56,Adomaa Memorial Hotel,KintampKintampo Tamale Road,8.050898,-1.732439
30,ru,tavda,58.0425,65.2726,-8.50,-8.50,87,0,5.86,The hotel is number 1,"Ulitsa Lenina, 28а, Tavda",58.051803,65.251526
33,gl,qaanaaq,77.4840,-69.3632,-1.86,-1.86,75,0,4.38,Qaanaaq Hotel,Qaanaaq,77.469287,-69.231591
52,no,eidsvag,62.7767,8.0668,18.50,21.20,79,0,5.75,Eidsvåg Fjordhotell,"Baksidevegen 116, Eidsvåg i Romsdal",62.768237,8.064150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,uy,rocha,-34.4833,-54.3333,54.30,54.30,81,0,3.09,Beleza Pura B&B,"Calle 12, 10, Balneario Buenos Aires",-34.489987,-54.321992
682,tv,asau,46.4333,26.4000,27.25,30.00,93,0,8.05,Casa Mika Ciobanus,"sat Ciobanus nr 103, Asău",46.443243,26.340587
690,et,mega,4.0500,38.3000,74.16,74.16,26,0,18.75,Amanuel Guest House,Mega,4.058035,38.320213
692,ru,sladkovo,55.5284,70.3385,-11.04,-11.04,87,0,4.94,Uyutnaya,"Ленина, 53, Sladkovo",55.529644,70.351971


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lnt"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"{Hotel}" for Hotel in hotel_info]
)
# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))